Canopy Smax

In [2]:
# make_smax_canopy_simple.py
import numpy as np
import pandas as pd
import rasterio

# ---- inputs ----
landcover_path = "landcover_Quinuas.tif"
table_csv      = "landcover_props.csv"   # your CSV
out_path       = "Smax_canopy.tif"       # output (mm)
k_extinction   = 0.4                     # LAI = -ln(1 - Cover)/k

# ---- read land-cover raster ----
with rasterio.open(landcover_path) as src:
    lc = src.read(1, masked=True)
    profile = src.profile

# ---- read only 6 columns & 12 rows from CSV, and clean ----
usecols = [0, 1, 2, 3, 4, 5]  # Description, Nr, RR[cm], N, Plant height[m], Cover
df = pd.read_csv(
    table_csv,
    usecols=usecols,
    nrows=12,
    engine="python",
    on_bad_lines="skip"
)

# standardize column names regardless of exact header text
df.columns = ["Description", "Nr", "RR_cm", "N", "PlantHeight_m", "Cover"]

# coerce types and sanitize
df["Nr"]    = pd.to_numeric(df["Nr"], errors="coerce")
df["Cover"] = pd.to_numeric(df["Cover"], errors="coerce").fillna(0.0).clip(0.0, 0.999999)
df = df.dropna(subset=["Nr"]).copy()
df["Nr"] = df["Nr"].astype(int)

# ---- map each class (Nr) to a vegetation type (no veg_type column needed) ----
# Adjust these if your species differ. (Based on your class list.)
vegtype_by_nr = {
    3:  "broadleaf",   # Forest formation (use 'pine' or 'eucalypt' if you know it)
    9:  "pine",    # Silviculture/Plantation (change to 'pine' or 'broadleaf' as needed)
    11: "bracken",     # Non-forest wetland (shrub/herb proxy; switch to 'grass' if herbaceous)
    12: "grass",       # Grassland
    13: "bracken",     # Non-forest natural / scrub
    21: "crop",        # Mosaic of farming uses
    25: "none",        # Anthropic non-vegetated
    29: "none",        # Rocky outcrop
    30: "none",        # Mining
    33: "none",        # Water bodies
    0:  "none"         # (if 0 occurs)
}

# build simple lookups
cover_lookup = dict(zip(df["Nr"], df["Cover"]))

# ---- compute LAI per pixel from Cover: LAI = -ln(1 - Cover)/k ----
classes = np.unique(lc.compressed()).astype(int)

cover = np.zeros(lc.shape, dtype="float32")
for cid in classes:
    cover[lc == cid] = float(cover_lookup.get(cid, 0.0))

LAI = -np.log1p(-cover) / float(k_extinction)

# ---- LAI -> Smax_canopy (mm) by vegetation type ----
def smax_from_lai(lai, vt):
    vt = (vt or "none").lower()
    if vt == "broadleaf": return 0.2856 * lai
    if vt == "pine":      return 0.2331 * lai
    if vt == "eucalypt":  return 0.0918 * (lai ** 1.04)
    if vt == "grass":     return 0.59   * (lai ** 0.88)
    if vt == "bracken":   return 0.1713 * lai
    if vt == "crop":      return 0.935 + 0.498*lai - 0.00575*(lai**2)
    return 0.0

smax = np.zeros(lc.shape, dtype="float32")
for cid in classes:
    vt = vegtype_by_nr.get(cid, "none")
    if vt == "none": 
        continue
    mask = (lc == cid)
    smax[mask] = smax_from_lai(LAI[mask], vt).astype("float32")

# ---- write output raster aligned to land-cover ----
profile.update(dtype="float32", count=1, compress="deflate")
with rasterio.open(out_path, "w", **profile) as dst:
    dst.write(smax, 1)

print(f"Wrote {out_path} (mm)")

# ---- compute & print average LAI per land-cover class ----
lai_stats = []
for cid in classes:
    mask = (lc == cid)
    if mask.sum() == 0:
        continue
    mean_lai = float(np.mean(LAI[mask]))
    lai_stats.append({"Class": int(cid), "Mean_LAI": mean_lai})

lai_df = pd.DataFrame(lai_stats).sort_values("Class")
print("\nAverage LAI per land-cover class:")
print(lai_df.to_string(index=False))


Wrote Smax_canopy.tif (mm)

Average LAI per land-cover class:
 Class  Mean_LAI
     0 34.505795
     3  5.756463
     9  4.023594
    11  5.756462
    12 34.505795
    13 34.505795
    21  5.756462
    25  0.000000
    29  0.000000
    30  0.000000
    33  5.756462


S_max surface

In [11]:
# make_smax_surface_fixA.py
# - Computes slope (%) in METERS even if landcover grid is geographic (deg)
# - Builds Smax_surface (mm) from RR (cm) per class and slope (%)

import math
import numpy as np
import pandas as pd
import rasterio
from rasterio.warp import reproject, Resampling

# ---------- USER INPUTS ----------
landcover_path = "landcover_Quinuas.tif"   # categorical map
dem_raw_path   = "DEM_Quinuas.tif"         # DEM in meters (any CRS)
table_csv      = "landcover_props.csv"     # columns: Nr, RR [cm] (or RR_cm)

# ---------- CODE OUTPUTS ----------
slope_out      = "slope_percent.tif"
smax_out       = "Smax_surface.tif"

# ---------- LOAD LANDCOVER (TARGET GRID) ----------
with rasterio.open(landcover_path) as lc_src:
    lc = lc_src.read(1, masked=True)
    lc_profile   = lc_src.profile
    dst_crs      = lc_src.crs
    dst_transform= lc_src.transform
    dst_width, dst_height = lc_src.width, lc_src.height

# ---------- LOAD DEM AND WARP TO LANDCOVER GRID ----------
with rasterio.open(dem_raw_path) as dem_src:
    dem_arr = np.full((dst_height, dst_width), np.nan, dtype=np.float32)
    reproject(
        source=rasterio.band(dem_src, 1),
        destination=dem_arr,
        src_transform=dem_src.transform,
        src_crs=dem_src.crs,
        dst_transform=dst_transform,
        dst_crs=dst_crs,
        dst_nodata=np.nan,
        resampling=Resampling.bilinear,
    )

dem = np.ma.masked_invalid(dem_arr)

# ---------- COMPUTE SLOPE IN PERCENT (FIX A: do math in METERS) ----------
# Get pixel size from target transform
px_deg = abs(dst_transform.a)  # pixel width (could be degrees)
py_deg = abs(dst_transform.e)  # pixel height (positive magnitude)

# Estimate raster center latitude (for degree->meter conversion)
# y = d*col + e*row + f; with no rotation d~0, so:
row_c = dst_height / 2.0
col_c = dst_width  / 2.0
y_center = dst_transform.d * col_c + dst_transform.e * row_c + dst_transform.f

# Convert horizontal spacing to METERS if CRS is geographic
if dst_crs is not None and dst_crs.is_geographic:
    # meters/degree at center latitude
    lat = y_center
    # Approx formulas for length of 1 degree (m)
    m_per_deg_lat = 111132.92 - 559.82*math.cos(2*math.radians(lat)) \
                    + 1.175*math.cos(4*math.radians(lat)) \
                    - 0.0023*math.cos(6*math.radians(lat))
    m_per_deg_lon = 111412.84*math.cos(math.radians(lat)) \
                    - 93.5*math.cos(3*math.radians(lat)) \
                    + 0.118*math.cos(5*math.radians(lat))
    px_m = px_deg * m_per_deg_lon
    py_m = py_deg * m_per_deg_lat
else:
    # already metric
    px_m = px_deg
    py_m = py_deg

# Guard against zeros
px_m = max(px_m, 1e-6)
py_m = max(py_m, 1e-6)

# Central differences with spacing in METERS
dz_dy, dz_dx = np.gradient(dem.filled(np.nan), py_m, px_m)  # rows->y, cols->x
slope_pct = 100.0 * np.hypot(dz_dx, dz_dy)
slope_pct = np.ma.masked_invalid(slope_pct)

# ---------- WRITE SLOPE RASTER ----------
slope_profile = lc_profile.copy()
slope_profile.update(dtype="float32", count=1, compress="deflate", nodata=np.float32(-9999))
with rasterio.open(slope_out, "w", **slope_profile) as dst:
    dst.write(slope_pct.filled(-9999).astype("float32"), 1)

# ---------- READ RR FROM CSV (ONLY NEEDED COLUMNS) ----------
raw = pd.read_csv(table_csv, nrows=12, dtype=str, engine="python", on_bad_lines="skip")
raw.columns = [c.strip() for c in raw.columns]

def find_col(cands):
    for c in raw.columns:
        if c.lower() in [x.lower() for x in cands]:
            return c
    raise KeyError(f"Column not found; tried {cands}")

nr_col = find_col(["Nr"])
rr_col = find_col(["RR [cm]", "RR_cm", "RR cm", "Random Roughness cm"])

df = raw[[nr_col, rr_col]].copy()
df["Nr"]    = pd.to_numeric(df[nr_col], errors="coerce")
df["RR_cm"] = pd.to_numeric(df[rr_col], errors="coerce")
df = df.dropna(subset=["Nr"]).fillna({"RR_cm": 0.0})
df["Nr"] = df["Nr"].astype(int)

# lookup: class id -> RR in mm (cm -> mm)
rr_lookup = {int(r.Nr): float(r.RR_cm) * 10.0 for r in df.itertuples(index=False)}

# ---------- MAP RR AND COMPUTE Smax_surface ----------
RRmm = np.zeros(lc.shape, dtype="float32")
classes = np.unique(lc.compressed()).astype(int)
for cid in classes:
    RRmm[lc == cid] = float(rr_lookup.get(cid, 0.0))

S = slope_pct.filled(0).astype("float32")  # slope in percent
Smax_surface = 0.243*RRmm + 0.010*(RRmm**2) + 0.012*RRmm*S

# Mask where DEM was invalid
Smax_surface = np.where(np.isnan(dem_arr), np.float32(-9999), Smax_surface).astype("float32")

# ---------- WRITE Smax_surface ----------
smax_profile = lc_profile.copy()
smax_profile.update(dtype="float32", count=1, compress="deflate", nodata=np.float32(-9999))
with rasterio.open(smax_out, "w", **smax_profile) as dst:
    dst.write(Smax_surface, 1)

print(f"Wrote {slope_out} (slope in percent)")
print(f"Wrote {smax_out} (Smax_surface in mm)")


Wrote slope_percent.tif (slope in percent)
Wrote Smax_surface.tif (Smax_surface in mm)
